# Bank Project - Preprocessing

In [316]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import preprocessing as prep
from sklearn.compose import ColumnTransformer

## Fetch Data

In [310]:
df = pd.read_csv('./data/bank-full.csv', sep=';')

In [311]:
cat_features = ['job','marital','education','default','housing','loan','contact','poutcome','month']

In [312]:
ct = ColumnTransformer(
    [
    ("logplus1",LogPlus1Transformer(),['duration','age']),
    ("campaign",CampaignTransformer(),'campaign'),
    ('ohe',OneHotEncoder(sparse=False),cat_features)
    ]
    )

df_trans = ct.fit_transform(df)

In [313]:
pd.DataFrame(df_trans).head()

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,5.56834,4.07754,"(0.999, 2.0]",0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
1,5.02388,3.80666,"(0.999, 2.0]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,4.34381,3.52636,"(0.999, 2.0]",0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,4.5326,3.8712,"(0.999, 2.0]",0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,5.2933,3.52636,"(0.999, 2.0]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
